# (Deng, Li and Guo, 2014) Statistical Inference in Two-Stage Online Controlled Experiments with Treatment Selection and Validation

[Link](https://www.exp-platform.com/Documents/p609-deng.pdf)

# Abstract

A/B Testing support decision making by directly comparing two variants at a time.
1. Two candidate treatments
2. a candidate treatment and an established control

In practice, one typically runs an experiment with multiple treatments together with a control to make decision for both purposes simultaneously.

There are two issues with this:
1. having multiple treatments increases false positives
2. selection process causes an upward bias in estimated effect size of the best observed treatment

To overcome these issues, a two-stage process is recommended, in which we select the best treatment from the first screening stage and then run the same experiment with only the selected best treatment and the control in the validation stage.

Traditional application of this two-stage design focus only on results from the second stage.

In this paper, we propose a general methodology for combining the first screening stage data together with validation stage data for more sensitive hypothesis testing and more accurate point estimation of the treatment effect.

Our method is widely applicable to existing online controlled experimentation systems.

# 1. Introduction

When there are multiple shipping candidates, we design the screening stage experiment to select the most promising one. If such a best candidate exists, we conduct a second validation run to make final ship decision and measure treatment effect. This type of two stage experiments with treatment selection of validation is commonly used in practice.

The space of treatment candidates ranges from 2 to 5 or even 10 in the screening stage. When candidate number exceeds 10, we can aggreisvely sift candidates via offline measurement or "paired test" such as interleaving (Radlinksi and Craswell 2013) to boost statistical power in the data analysis.

The validation stage involves only winner from the screening stage and control. It is analyzed in the traditional A/B testing framework and is well-understood. The first screening stage, however, includes simultaneous analysis of multiple treatments. We need to adjust hypothesis testing procedure to control for inflated false positives in multiple comparison (Section 2.2). We improve on traditional adjustments such as Bonferroni and Holm's method (Holm 1979), as they are typically too conservative.

In Section 3 we propose a sharp adjustment method that is exact in the sense that it touches the claimed Type I error.

Point estimation is also nontrivial as the treatment selection introduces an upward bias (Lemma 4).

One might wonder why this is important since in A/B testing people are generally only interested in finding the best candidate to ship. We found in a data driven organization it is equally crucial to keep accurate records of impacts made by each individual feature.

In Section 4 we propose several methods to correct the bias and investigate more efficient estimators by combining data from bot stages.

In Section 4, we show an insightful theoretical result to ensure we can almost always treat the treatment effect estimates from two stages as independent, given treatment procedures which assign independently, despite overlap of experiment subjects in the two stages.

# 2. Background and Weak Dependence between Estimates in Two Stages

Here we cover the analysis of one stage test, using notation from (Deng et al. 2013).



## 2.1 Treatment vs Control in One Stage A/B Test

Suppose we are interested in a metric $X$ (e.g., Clicks per user).

Assume the observed values of the metric for users in the treatment and control are independent realizationis of random variables $X^{(t)}$ for treatment and $X^{(c)}$ for control respectively, we can apply the t-test to determine if the diff. between the two groups is statistically significant.

Under the random user effect model, for user $i$ in control group,

$$X_i^{(c)}=\mu+\alpha_i+\epsilon_i$$

where:
- $\mu$ is the mean of $X^{(c)}$
- $\alpha_i$ represents uesr random effect
- $\epsilon_i$ is random noise

Random user effect $\alpha$ has mean 0 and variance $\text{Var}(\alpha)$. Residual $E[\epsilon|\alpha]=0$. The random pair $(\alpha_i,\epsilon_i)$ are i.i.d for all users.

We don't assume independence of $\epsilon_i$ and $\alpha_i$, as the distribution of $\epsilon$ might depend on $\alpha$ (e.g., users who cilck more might also have larger random variation in their clicks). However they are uncorrelated by construction since $E(\epsilon\alpha)=E[E(\epsilon|\alpha)]=0$.

For treatment group,

$$X^{(t)}_i=\mu+\theta_i+\alpha_i+\epsilon_i$$

where fixed treatment effect $\theta_i$ can vary from user to user but $\theta$ is uncorrelated to the noise $\epsilon$.

The ATE is defined as the expectation of $\theta$. The null hypothesis is that $\delta:=E(\theta)=0$ and the alternative is that it is not $0$ for a two-sided test. For one-sided test, the alternative is $\delta\leq 0$ (looking for positive change) or $\delta\geq 0$ (looking for negative change).

The t-test is based on the t-statistic:

$$\frac{\bar X^{(t)}-\bar X^{(c)}}{\sqrt{\text{Var}(\bar X^{(t)}-\bar X^{(c)})}}$$

where observed difference between treatment and control $\Delta=\bar X^{(t)}-\bar X^{(c)}$ is an unbiased estimator for the shift of the mean and the t-statistic is a normalized version of that estimator.

In traditional t-tset (Student 1908), one needs to assume equal variance and normality of $\bar X^{(t)}$ and $\bar X^{(c)}$.

In practice, the equal variance assumption can be relaxed by using Welch's t-test (Welch 1947).

For online experiments with the sample sizes for both control and treatment at least in the thousands, even the normality assumption on $X$ is usually unnecessary. To see that, by central limit theorem, $\bar X^{(t)}$ and $\bar X^{(c)}$ are both asymptotically normal as the sample size $m$ and $n$ for treatment and control increases. $\delta$ is therefore approximately normal with variance

$$\text{Var}(\delta)=\text{Var}(\bar X^{(t)}-\bar X^{(c)})=\text{Var}(\bar X^{(t)})+\text{Var}(\bar X^{(c)})$$

The t-statistics is approximately standard normal so t-test in large sample case is equivalent to $z$-test.

The central limit theorem only assumes finite variance which almost always applies in online experimentation. The speed of convergence to normal can be quantified by using Berry-Essen theorem (DasGupta 2008). We have verified that most metrics we tested at Being are well approximated by normal distribution in experiments with thousands of samples.

## 2.2 Multiple Treatments in A/B Test

When there is only one treatment compared to a control, $\Delta$ is both the Maximum likelihood estimator (MLE) of the treatment effect, and an unbiased estimator. However this is not true for multiple treatments for $\max (\delta^{(j)})$ where $j$ is the $j$th treatment. Observe: when there are multiple treatments and we observe $\Delta^{(j)}$ for the $j$th treatment, we can see that $\max(\Delta^{(j)})$ is the MLE of $\max(\delta^{(j)}$, where $\delta^{(j)}$ is the true underlying treatment effect of the $j$th treatment. However, it is no longer an unbiased estimator because it skews positive (imagine a scenario where $\delta^{(j)}=0$ for all $j$)

Besides point estimation, hypothesis testing in experiments with multiple treatments also suffers from an issue called multiple comparison.

To deal with this, various p-value adjustment techniques have been proposed, such as Bonferroni correction, Holm's method (Holm 1979) and FDR-based methods suitable for even larger number of simultaneous comaprisons (Benjamini and Hochberg 1995). Both Bonferroni and Holm's method are applicable to the general case with unknown covariance structure between test statistics of all comparisons.

In the context of online A/B testing, when we have large samples, we live in a simplier multivariate normal world. We have full knowledge of the covariance structure of this multivariate normal distribution and we should be able to exploiot it to come up with a better hypothesis testing procedure. Section 3 contains more details.



## 2.3 Weak Dependence

When we combine results from two stages to form a more sensitive test and estimate treatment effect more accurately, one of the challenges we face is caused by possible dependence of the observed metric values from the two stages. In theory we may force independence between the two stages by running them on separate traffic, so the two stages share no users in common. This is undesirable in any scaled platform (Kohavi et al. 2013) because
1. It means the total traffic in both stages combined cannot exceed 100%, thus we suffer decreased power in both stages
2. requires additional infrastructure to ensure no overlap.

However, in practice we can safely assume independence between the observed $\Delta$ from two stages, as long as the randomization procedure used in the two stages are independent.

The first result of this paper (Theorem 1: Almost Uncorrelated Deltas) show that this is the case.


# 3. Hypothesis Testing

Combining results from diff. studies is the subject of a field called meta-analysis. In this section we present a method for hypothesis testing utilizing data from both stages. Later in Sec. 4 we use combined data for point estimation.

## 3.1 Meta-Analysis: Combine Data from Two Stages

Gives a formula for combining $p$-values. Explains how multiplying two $p$-values (in the case of 2 p-values) underestimates the true Type I error.

While the original formula for calculating the Type I error for more than 2 p-values is quickly cumbersome, Fisher (1970) noticed natural log function transforms uniform(0,1) distribution into an exponential(1) distribution and exponential(1) is half of a $\chi^2$ with 2 degrees of freedom. (remember, the null is that the p-value comes from a uniform dist.)

In this connection, the product of $k$ p-values under null hypothesis is a sum of independent exponential(1) and

$$2\log(\prod p_i)=\sum(2\log(p_i)\sim\chi^2_{2k}$$

This result is known as the Fisher's method and can be used to combine tests under the assumption of independent p-values.

It is also a model-free method in the sense that it only utilizes p-values without tapping into the distribution of the test statistics.

It is not surprising that in our cases, by using normality and known covariance structure of our observed $\Delta$'s, we should be able to get a more sensitive test.

We still have the multiple comparison issue to tackle. One standard method is Bonferroni correction. Specifically,
1. First we determine the p-value from the screening stage using a Bonferronni correction. If there is $k$ treatment candidates in the screening run, if $p_1$ is the smallest p-value, we just divide this p-value by $K$.
2. We use this value plus the p-value for the second stage and combine using Fisher's method.

This method provides a valid hypothesis testing for two-stage A/B testing with treatment selection and validation. We will call this BF method and set it as benchmark.



## 3.2 Sharp Multiple Comparison Adjustment

Here we improve BF method in two directions.

This will be called generealized weighted average method since the test statistic is in a form of weighted average.

Although Bonferroni is the simplest and most used for multiple comparison adjustment, it is often too conservative in online experiments. This is because by central limit theorem, we can safely assume all metrics to be approximately normal. More specifically, let $X_1,X_2,...,X_k$ be the observed metric values (e.g., clicks per user) for the $k$ teratments and $X_0$ be the value for control, we can estimate the variance of each and take these as known in our model. Moreover, the covariance between $\Delta_i,i=1,...,k$ can also be estimated. In this scenario of complete distributional information, we can use a generalized step-down procedure (Romano 2005, Section 9.1, p.352)





# 4. Point Estimation

Another task for A/B tseting is to provide good estimation of the true treatment effects in terms of minimal mean squared error (MSE) that achieves a balance between bias and variance.

In the screening stage of the experiment, suppose we have $k$ different treatments with metric values $X_1,...,X_k$ respectively. Thanks to CLT, we can assume $\mathbb{X}=(X_1,X_2,...,X_k)^T\sim\mathcal{N}(\mu,\Sigma)$ where $mu=(\mu_1,...,\mu_k)^T$ and $\Sigma=\text{diag}(\sigma_1^2,...,\sigma_k^2)^2$. Moreover, there is a control group with the metric value $X_0\sim\mathcal{N}(\mu_0,\sigma_0^2)$. The estimation of $\sigma_i$ is easy to achieve with a large sample and of no interest in this paper, thus we assume the variances are known and fixed. Without loss of generality, assume $\sigma=\sigma_0=\cdot\cdot\cdot=\sigma_k$. Then we can use $\Delta_i=X_i-X_0$ as the estimation of the effect of the $i$th treatment.

At the end of the screening stage, we choose the treatment with the largest $\Delta$ then run the second-stage with only the control group and the largest $\Delta$.